In [40]:
import pandas as pd
import numpy  as np
import re                              # Used for splitting the Items with multiple delimiters
from   collections import Counter      # To count items in dictionaries
from   itertools   import combinations # To generate combinations of items
from   itertools   import permutations # To generate permutations of items
import functools
from collections import defaultdict  

routeID        = -1 # initialized to -1 in order to force the first execution to the "else"
orderID        = -1
tmp = [None] * 15

# The dataset was created with the notebook "DatasetCreation.ipynb".

routes = pd.read_csv('route_trips.csv', 
                     delimiter = ',', 
                     encoding  = 'utf-8')

routes.drop(routes.columns[0], 
            axis    = 1, 
            inplace = True)

In [41]:
# Prepare the delimiters for splitting the items
delimiters   = ",", "{", "}", "'"
regexPattern = '|'.join(map(re.escape, delimiters))

In [42]:
# Produce a new column for the Items
newItemsColumn = []
for index in range(0, len(routes)):
    tmp = list(filter(None, re.split(regexPattern, routes.Items[index])))
    newItemsColumn.append([item for item in tmp 
                                   if item.strip()])
routes.Items = newItemsColumn

In [43]:
# Save in a dictionary the number of trip for each order 
routeDimension = defaultdict(list)
for route in routes.RouteId:
    routeDimension[route] = max([order for order in routes[routes.RouteId == route].Order]) + 1

In [44]:
# Set the threshold for the support of Frequent itemset theory
supportThreshold = 5

items   = [item for items in routes.Items 
                   for item in items]

# Count the occurrencies of each item in the whole dataset
# ("items" now contains a list of all the products for each trip)
itemCount = Counter([(item,) for item in items])
support = itemCount

# Select only those items that appear more than the support
frequentItems = [item for item in itemCount 
                    if itemCount[item] >= supportThreshold]

In [45]:
routes.Items

0      [Goat-milk-cheese, Kartoffelchips, Chocolate-b...
1      [Goat-milk-cheese, Whole-bean-coffee, Nectar-d...
2      [Soupes, Whole-bean-coffee, Chocolate-block, M...
3      [Orange-chocolate, Seabass, Soupes, Whole-bean...
4      [Citrus-punch, Chocolate-block, Pork-chipolata...
5      [Bircarbonate-of-soda, Flavoured-cranberries, ...
6      [Apple-blackcurrant-drink, Strawberry-trifle, ...
7      [Ratatouille, Cheddar-baguettes, Fruit-spread,...
8      [Cheddar-baguettes, Canneloni, Green-kale, Pea...
9      [Strawberry-trifle, Nans, Canneloni, Pickles, ...
10     [Custard-tarts, Prunes, Galettes-de-polenta, R...
11     [Fruchtnektar, Custard-tarts, Prunes, Pitted-p...
12     [Baking, Custard-tarts, Chutney, Pitted-prunes...
13     [Susswaren, Onion-relish, Peanut-in-chocolate,...
14     [Peanut-in-chocolate, Seitan-bio, Burrito-kit,...
15     [Rice-candy, Peanut-in-chocolate, Cold-meats, ...
16     [Canned-precooked-meat, Malt-vinegar, Pitted-p...
17     [Danette-chocolate-desse

In [46]:
[f for ft in frequentItems for f in ft]

['Goat-milk-cheese',
 'Kartoffelchips',
 'Chocolate-block',
 'Pate-a-tartiner-pomme-poire',
 'Organic-spelt-flour',
 'Plats-prepares',
 'Whipping-fresh-cream',
 'Confiseries',
 'Couscous',
 'Juice-milk-drink',
 'Formulated-supplementary-food',
 'Lime-pickle',
 'Sandiwch',
 'Whole-bean-coffee',
 'Nectar-de-fruit',
 'Orange-chocolate',
 'Preparation-for-dessert',
 'Hommus',
 'Soupes',
 'Corn-tortilla',
 'Graines',
 'Redcurrant-tea',
 'Mango',
 'Levure',
 'Vegetarian-mock-meat',
 'Seed-style-mustard',
 'Multivitamin-supplement',
 'Kiwi-fruit',
 'Pralines',
 'Oatcakes',
 'Seabass',
 'Waffles',
 'Breakfast-biscuits',
 'Nutella-biscuits',
 'Choux-fleurs-en-fleurettes-surgeles',
 'Milk substitute',
 'Citrus-punch',
 'Pork-chipolatas',
 'Bircarbonate-of-soda',
 'Flavoured-cranberries',
 'Pumpkin-soup',
 'Fruit-spread',
 'Rice-flakes-sheet',
 'Cones',
 'Indian-sauces',
 'Strawberry-trifle',
 'Mulled-wine',
 'Sourdough-bread',
 'Purple-sprouting-broccoli',
 'Mate',
 'Seafood',
 'Artisan-bread-mi

In [47]:
# Now we remove from the Items those that are not frequentItems
routes.Items = [list(filter(lambda i: i in [f for ft in frequentItems for f in ft], items)) for items in routes.Items]

In [48]:
routes.Items

0      [Goat-milk-cheese, Kartoffelchips, Chocolate-b...
1      [Goat-milk-cheese, Whole-bean-coffee, Nectar-d...
2      [Soupes, Whole-bean-coffee, Chocolate-block, C...
3      [Orange-chocolate, Seabass, Soupes, Whole-bean...
4      [Citrus-punch, Chocolate-block, Pork-chipolata...
5      [Bircarbonate-of-soda, Flavoured-cranberries, ...
6      [Apple-blackcurrant-drink, Strawberry-trifle, ...
7      [Ratatouille, Cheddar-baguettes, Fruit-spread,...
8      [Cheddar-baguettes, Canneloni, Green-kale, Pea...
9      [Strawberry-trifle, Nans, Canneloni, Custard-t...
10     [Custard-tarts, Prunes, Galettes-de-polenta, R...
11     [Fruchtnektar, Custard-tarts, Prunes, Pitted-p...
12     [Baking, Custard-tarts, Chutney, Pitted-prunes...
13     [Susswaren, Onion-relish, Peanut-in-chocolate,...
14     [Peanut-in-chocolate, Seitan-bio, Burrito-kit,...
15     [Rice-candy, Peanut-in-chocolate, Senate-bean-...
16     [Canned-precooked-meat, Pitted-prunes, Senate-...
17     [Danette-chocolate-desse

In [49]:
# Here we generate combinations of 2 frequent items 
# and we count how many pairs there are for each combination
pairCount     = Counter([comb for combs in [list(permutations(items, 2)) for items in routes.Items] 
                                 for comb in combs])
support = support + pairCount

# and select only those that appear more than the support
frequentPairs = [comb for comb in pairCount 
                         if pairCount[comb] >= supportThreshold]

In [50]:
# then sort them for next operations and reduce them to a set for removing duplicates  
frequentPairs = list(set( [ tuple(fp) for fp in [sorted(fp) for fp in frequentPairs ] ] ) )

In [51]:
# Here we remove the duplicates from the frequentPairs
itemInPairs = set(f for fp in frequentPairs for f in fp)

In [52]:
# This is used to keep track of the tuples at the i-th step
itemsithStep = [list(filter(lambda i: i in itemInPairs, items)) for items in routes.Items]

In [53]:
#def Remove(duplicate): 
#    final_list = [] 
#    for num in duplicate: 
#        if num not in final_list: 
#            final_list.append(num) 
#    return final_list 

In [ ]:
# Max depth for frequent tuples
#depth = 6
#frequentTuples = []
#for i in range(2,depth):
#    subset = []
#    for sl in range(0,len(frequentPairs)):
#        tryitems = frequentPairs[sl][:-1]
#        possibleitems = []
#        for asl in frequentPairs[:sl]+frequentPairs[sl+1:]:
#            if (len(set(tryitems).intersection(set(asl))) == len(tryitems)):
#                possibleitems.append(list(set(asl).difference(set(tryitems))))
#        if possibleitems != []:
#            for ssl in list(combinations(frequentPairs[sl], len(frequentPairs[sl])-1)):
#                if ssl != tryitems:
#                    for el in possibleitems:
#                        elem = el
#                        elem.extend(ssl)
#                        c = tuple(set(elem))
#                        if(set(c) in [set(fp) for fp in frequentPairs]):
#                            c = list(c)
#                            c.extend(list(item for item in tryitems))
#                            subset.append(tuple(set(c)))
#    subset = [tuple(s) for s in Remove(list(set(sb) for sb in subset))]
#    combination = [list(combinations(items, i+1)) for items in routes.Items]
#    comb = [tuple(comb) for combs in combination 
#                for comb in combs]
#    tuplesCount = Counter(comb)
    # and select only those that appear more than the support
#    frequentT = [tuples for tuples in tuplesCount 
#                        if tuplesCount[tuples] >= support]
#    frequentTuples.append(frequentT)
#    frequentPairs = frequentT

In [54]:
# Max depth for frequent tuples
depth            = 6
frequentTuples   = []
for i in range(2, depth):
    subset       = []
    combination  = [list(permutations(items, i + 1)) for items in itemsithStep]
    comb         = [tuple(comb) for combs in combination 
                                   for comb in combs]
    tuplesCount  = Counter(comb)
    support = support + tuplesCount
    frequentT    = [tuples for tuples in tuplesCount 
                            if tuplesCount[tuples] >= supportThreshold]
    frequentTuples.append(frequentT)
    itemInTuples = set(f for fp in frequentT for f in fp)
    itemsithStep = [list(filter(lambda i: i in itemInTuples, items)) for items in itemsithStep]

In [55]:
# popularTuples will be our container for interesting objects. 
# Now we can find inside popular items and popular pairs but it is going to be extended.
popularTuples = [ft for ft in frequentItems]
popularTuples.extend(frequentPairs)
for ft in frequentTuples:
    popularTuples.extend(ft)

In [56]:
def matches(rid, order, popularTuples):
    itemList   = []
    orderItems = routes.Items[(routes.RouteId == rid) & 
                              (routes.Order   == order)
                             ].tolist()[0]
    for obj in popularTuples:
        check = True # here we check if none of the object is inside the orderItems
        for item in obj:
            if item not in orderItems:
                check = False
                break
        if check:    # in that case we add it to the itemList
            itemList.append(list(obj))
    return itemList

def anyOrder(rid, oid, start, popularTuples):
    global orderID
    global tmp
    t = []
    if tmp[oid] == None:
        t = matches(rid, oid, popularTuples)
        tmp[oid] = t
    else:
        t = tmp[oid]
    
    return tuple([oid - start - 1, t])
    
def divideOrder(rid, order, popularTuples):
    global orderID
    global routeID
    global tmp
    dim = routeDimension[rid]
    if (order + 1 < dim):
        if(rid != routeID):
            routeID = rid
            orderID = order
            tmp = [None] * 15
        return [anyOrder(rid, oid, order, popularTuples) for oid in range(order + 1, dim)]
    else:
        return [(-1, [])]

def searchNext(tuples, popularTuples):
    keys        = [(routes.RouteId[row], routes.Order[row]) 
                 for row in range(0, len(routes)) 
                     if set(tuples).intersection(set(routes.Items[row])) == set(tuples)]
    newTuples   = [divideOrder(key[0], key[1], popularTuples) for key in keys]
    return newTuples

In [57]:
import time
a = time.time()
popularPatterns = [(pop, searchNext(pop, popularTuples)) for pop in popularTuples]
print(time.time()-a)

48.448314905166626


In [58]:
# Confidence and Interest level
confidence = 0.5
interest = 0.5

def patternSum(pattern):
    finalPattern = [(i, sum(val for val in pattern[i])) for i in pattern]
    return finalPattern
        
def patternReduce(pattern):
    d = defaultdict(list)
    for key, value in pattern:
        t = tuple([key[0], tuple(key[1])])
        d[t].append(value)
    return d

def patternCount(pattern):
    reduction = patternReduce(pattern)
    count = patternSum(reduction)
    return count

# Limit of classification for frequent Patterns
def patternExtraction(pattern, limit = 5):
    patternLevel = [[(x[0], y), 1] for pat in pattern[1] for x in pat for y in x[1]]
    patternLevel = patternCount(patternLevel)
    finalPattern = [pat for pat in patternLevel if ((pat[1]/support[pattern[0]]) >= confidence) &
                                                   (abs((pat[1]/support[pattern[0]] -
                                                      (support[pat[0][1]]/len(routes)))) >= interest)] 
    return (pattern[0], finalPattern)

In [59]:
patterns = [patternExtraction(pattern) for pattern in popularPatterns]

In [60]:
patterns

[(('Goat-milk-cheese',), []),
 (('Kartoffelchips',), []),
 (('Chocolate-block',), [((0, ('Chocolate-block',)), 9)]),
 (('Pate-a-tartiner-pomme-poire',), []),
 (('Organic-spelt-flour',), []),
 (('Plats-prepares',), []),
 (('Whipping-fresh-cream',), []),
 (('Confiseries',), []),
 (('Couscous',), []),
 (('Juice-milk-drink',), []),
 (('Formulated-supplementary-food',), []),
 (('Lime-pickle',), []),
 (('Sandiwch',), []),
 (('Whole-bean-coffee',), []),
 (('Nectar-de-fruit',), []),
 (('Orange-chocolate',), []),
 (('Preparation-for-dessert',), []),
 (('Hommus',), []),
 (('Soupes',), []),
 (('Corn-tortilla',), []),
 (('Graines',), []),
 (('Redcurrant-tea',), []),
 (('Mango',), []),
 (('Levure',),
  [((0, ('Levure',)), 5),
   ((0, ('Fresh-bread',)), 5),
   ((0, ('Fresh-bread', 'Levure')), 5)]),
 (('Vegetarian-mock-meat',), []),
 (('Seed-style-mustard',), []),
 (('Multivitamin-supplement',), []),
 (('Kiwi-fruit',), []),
 (('Pralines',), []),
 (('Oatcakes',), []),
 (('Seabass',), []),
 (('Waffles'

In [228]:
#stampo solo quelli che poi hanno dei livelli oltre all'elemento corrente
for p in patterns:
    c = 0
    if p[1] != []:
        print("-> ", p[0][0])
        for item in p[1]:
            s = "\t"
            if item != []:
                p = ""
                for i in range(0, item[0][0]):
                    s += "\t"
                for k in range(0,len(item[0][1])):
                    if k == (len(item[0][1])-1):
                        p += "\t"+item[0][1][k]
                    else:
                        p += "\t"+item[0][1][k] + ","
                print(s, item[0][0],"->", p)
        print()

->  Oatcakes
	 0 -> 	Oatcakes

->  Flavouring-extract
	 0 -> 	Flavouring-extract

->  Prepared-salads
	 0 -> 	Prepared-salads

->  Quinoa-and-rice
	 0 -> 	Quinoa-and-rice

->  Frozen-spring-rolls
	 0 -> 	Frozen-spring-rolls

->  Peanut-butter-cup
	 0 -> 	Peanut-butter-cup

->  Caramel-dessert
	 0 -> 	Baguettes-de-pain
	 0 -> 	Mayonaise
	 0 -> 	Caramel-dessert

->  Baguettes-de-pain
	 0 -> 	Baguettes-de-pain

->  Pays-d-oc
	 0 -> 	Baguettes-de-pain
	 0 -> 	Pays-d-oc
	 0 -> 	Prunes
	 0 -> 	Multifruits
	 0 -> 	Caramel-dessert

->  Prunes
	 0 -> 	Prunes

->  Mayonaise
	 0 -> 	Mayonaise

->  Carots
	 0 -> 	Carots

->  Chopped-tomatoes
	 0 -> 	Chopped-tomatoes
	 0 -> 	Chopeed-tomatoes
	 0 -> 	Chopeed-tomatoes,	Chopped-tomatoes

->  Pizza-bases
	 0 -> 	Pizza-bases
	 0 -> 	Schwarztees
	 0 -> 	Nectar-de-pommes

->  Cornbreads
	 0 -> 	Cornbreads

->  Fudge
	 0 -> 	Fudge

->  Bitters
	 0 -> 	Bitters

->  Tigernuts
	 0 -> 	Tigernuts

->  Aperitif
	 0 -> 	Aperitif

->  Beef-mince
	 0 -> 	Beef-mince

	 0 -> 	Cafes-solubles

->  Malt-vinegar
	 0 -> 	Sun-dried-tomatoes
	 0 -> 	Malt-vinegar

->  Distilled-malt-vinegar
	 0 -> 	Chicken-breast-curry-with-tomato-spinach-sauce
	 0 -> 	Distilled-malt-vinegar
	 0 -> 	Chicken-breast-curry-with-tomato-spinach-sauce,	Distilled-malt-vinegar
		 1 -> 	Chicken-breast-curry-with-tomato-spinach-sauce

->  Fecules
	 0 -> 	Asian-cooking
		 1 -> 	Asian-cooking

->  Cured-sausage
	 0 -> 	Cured-sausage

->  Pasteurized-milk
	 0 -> 	Pasteurized-milk

->  Pale-ale
	 0 -> 	Pale-ale

->  Soup-mix
	 0 -> 	Milch
	 0 -> 	Raspberry-conserve
	 0 -> 	Dairies
	 0 -> 	Curries
	 0 -> 	Soup-mix
	 0 -> 	Milch,	Raspberry-conserve
	 0 -> 	Curries,	Raspberry-conserve

->  Organic-certified
	 0 -> 	Organic-certified

->  Pate-a-tartiner-pomme-poire
	 0 -> 	Pate-a-tartiner-pomme-poire
	 0 -> 	Fruit-sticks

->  Kaffee
	 0 -> 	Kaffee
		 1 -> 	Kaffee
			 2 -> 	Kaffee

->  Cordial
	 0 -> 	Cordial

->  Ramen
	 0 -> 	Ramen

->  Pies
	 0 -> 	Fruit-spreads
	 0 -> 	Pies
	 0 -> 	Multi

	 0 -> 	Food additives
	 0 -> 	Fish-sauce
	 0 -> 	Fish-sauce,	Food additives
	 0 -> 	Fish-sauce,	Sucre
	 0 -> 	Food additives,	Sucre
	 0 -> 	Food additives,	Fish-sauce,	Sucre
	 0 -> 	Food additives,	Sucre,	Fish-sauce
	 0 -> 	Fish-sauce,	Food additives,	Sucre
	 0 -> 	Fish-sauce,	Sucre,	Food additives
	 0 -> 	Sucre,	Food additives,	Fish-sauce
	 0 -> 	Sucre,	Fish-sauce,	Food additives
		 1 -> 	Sucre
		 1 -> 	Food additives
		 1 -> 	Fish-sauce
		 1 -> 	Fish-sauce,	Food additives
		 1 -> 	Fish-sauce,	Sucre
		 1 -> 	Food additives,	Sucre
		 1 -> 	Food additives,	Fish-sauce,	Sucre
		 1 -> 	Food additives,	Sucre,	Fish-sauce
		 1 -> 	Fish-sauce,	Food additives,	Sucre
		 1 -> 	Fish-sauce,	Sucre,	Food additives
		 1 -> 	Sucre,	Food additives,	Fish-sauce
		 1 -> 	Sucre,	Fish-sauce,	Food additives

->  Fish-sauce
	 0 -> 	Sucre
	 0 -> 	Food additives
	 0 -> 	Fish-sauce
	 0 -> 	Fish-sauce,	Food additives
	 0 -> 	Fish-sauce,	Sucre
	 0 -> 	Food additives,	Sucre
	 0 -> 	Food additives,	Fish-sauce,	Sucre

In [ ]:
from copy import deepcopy
p2 = deepcopy(patterns)#####################################da eliminare alla fine

In [ ]:
patterns = deepcopy(p2)#####################################da togliere alla fine
patNewLevel = []
for pattern in [p for p in patterns if p[1] != []]:
    l = [pattern[0][0],[]]
    for items in pattern[1]:
        level = items[0][0]
        item  = items[0][1]
        check = False
        for couple in l[1]:
            if couple[0] == level:
                couple[1].append(item)
                check = True
        if not check:
            l[1].append([level,[item]])
    patNewLevel.append(l)
patNewLevel = sorted(patNewLevel)

In [ ]:
# Return the length of a model (in terms of depth levels)
def howManyLevels(pattern):
    return len(pattern[1])

#Return the length of a route starting from the passed order
def howManyOrders(route, order): #remaining orders after the one selected
    length = 0
    before  = False
    check   = False
    end     = False
    index = 0
    while (not end) & (index < len(routes)):
        if routes.RouteId[index] == route:
            before = True
            if routes.Order[index] > order:     
                length += 1
        else:
            before = False
            
        if before == True:
            if check == False:
                check = True
        else:
            if check == True:
                end = True
        index += 1
    return length

# Return the sorted levels of a pattern
def getLevels(pattern):
    return sorted(pattern[1])

# Return the orders inside a route
def getRoute(routeId):
    route = []
    before  = False
    check   = False
    end     = False
    i = 0
    while (not end) & (i < len(routes)):
        if routes.RouteId[i] == routeId:
            before = True
            route.append(routes.iloc[i])
        else:
            before = False
        if before == True:
            if check == False:
                check = True
        else:
            if check == True:
                end = True
        i += 1
    return route

# Return the index in the dataframe of a couple (route, order)
def getIndex(route, order):
    for index in range(0, len(routes)):
        if (routes.RouteId[index] == route) & (routes.Order[index] == order):
            return index
    return -1

In [ ]:
import time
a = time.time()
foundInRoute = {}
for r in range(0, max(routes.RouteId)):
    for trip in getRoute(r):
        for pattern in patNewLevel:
            # If in this trip we have the head of the pattern
            if pattern[0] in trip.Items:
                # Go ahead only if there are at least the same number of orders and levels of the pattern
                if howManyLevels(pattern) <= howManyOrders(trip.RouteId, trip.Order): 
                    for level in getLevels(pattern):  
                        for item in level[1]:
                            if set(item).intersection(set(routes.Items[getIndex(trip.RouteId, 
                                                                                trip.Order) 
                                                                       + level[0] 
                                                                       + 1])) == set(item):
                                if foundInRoute.get((trip.RouteId, 
                                                     trip.Order), 
                                                    "ORDER") == "ORDER":
                                    foundInRoute[(trip.RouteId, trip.Order)] = {level[0]: set((item,))}
                                else:
                                    if foundInRoute.get((trip.RouteId, 
                                                         trip.Order)).get(level[0], 
                                                                          "LEVEL") == "LEVEL":
                                        foundInRoute[(trip.RouteId, 
                                                      trip.Order)][level[0]] = set((item,))
                                    else:
                                        foundInRoute.get((trip.RouteId,
                                                          trip.Order)).get(level[0]).add(item)
print()
print(time.time()-a)
print()
foundInRoute